<a href="https://colab.research.google.com/github/seenu-g/school_of_ai/blob/master/chap15/Load_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
cd '/content/gdrive/My Drive/school_of_ai/chap15/'

/content/gdrive/My Drive/school_of_ai/chap15


In [0]:
from torch.utils.data import Dataset, random_split
from PIL import Image
import numpy as np
import torch
import os
import torchvision.transforms as transforms
from tqdm import notebook

In [0]:
class RawDataSet(Dataset):
    def __init__(self,url):
        self.fg = []
        self.mask = []
        self.url = url
        self.bg = []
        
        for file_name in os.listdir(url+'Background/'):
            bg = Image.open(url+'Background/'+ file_name)
            bg_npimg = np.asarray(bg)
            
            if(len(bg_npimg.shape) ==2):  
                bg_npimg = np.repeat(bg_npimg[:, :, np.newaxis], 3, axis=2)
            self.bg.append(bg_npimg)  

        for file_name in os.listdir(url+'Foreground/'):
            fg = Image.open(url+'Foreground/'+ file_name)
            fg_npimg = np.asarray(fg)
            
            if(len(fg_npimg.shape) ==2):  
                fg_npimg = np.repeat(fg_npimg[:, :, np.newaxis], 3, axis=2)
            self.fg.append(fg_npimg)

        for file_name in os.listdir(url+'Fg-Mask/'):
            mask = Image.open(url+'Fg-Mask/'+ file_name)
            mask_npimg = np.asarray(mask)
            
            if(len(fg_npimg.shape) ==2):  
               mask_npimg = np.repeat(mask_npimg[:, :, np.newaxis], 3, axis=2)
            self.mask.append(mask_npimg)   

    def __len__(self):
        return len(self.fg)

    def __getitem__(self, idx):
        bg = self.bg[idx]
        fg = self.fg[idx]
        mask = self.mask[idx]
        return bg,fg,mask

In [0]:
  raw_folder = '/content/gdrive/My Drive/school_of_ai/archive/'
  raw_dataset = RawDataSet(raw_folder)
  print(len(raw_dataset))
  raw_dataloaders = torch.utils.data.DataLoader(raw_dataset)
  raw_dataiter = iter(raw_dataloaders)
  bg_images,fg_images,mask_images = raw_dataiter.next()
  bg_images = bg_images.numpy()
  fg_images = fg_images.numpy()
  mask_images = mask_images.numpy()
  print(bg_images.shape)
  print(fg_images.shape)
  print(mask_images.shape)

100
(1, 224, 224, 3)
(1, 90, 33, 4)
(1, 120, 45, 3)


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
import zipfile
import os.path
for i in range(1):
  if(os.path.isfile(f'/content/gdrive/My Drive/school_of_ai/archive/data_part{str(i+1)}.zip')):
    print(f'Extract from data_part{str(i+1)}.zip to folder data_{str(i+1)}')
    archive = zipfile.ZipFile(f'/content/gdrive/My Drive/school_of_ai/archive/data_part{str(i+1)}.zip')
    for file in archive.namelist():
        archive.extract(file, '/content/')

Extract from data_part1.zip to folder data_1


In [0]:
def get_images(url, folder) :
  images =[]
  if(os.path.exists(url+folder+'/')==True) :
       print("Loading "+ url+ folder+'/')
       for file_name in os.listdir(url+folder+'/'):
            img = Image.open(url+folder+'/'+ file_name)
            npimg = np.asarray(img)
                
            if(len(npimg.shape) ==2):  
                npimg = np.repeat(npimg[:, :, np.newaxis], 3, axis=2)
            images.append(npimg)
  else:
       print("Path does not exisit",url+folder+'/')
  return images; 

class InputDataSet(Dataset):
    def __init__(self,url, folder="all"):
        self.fg_bg = []
        self.fg_bg_mask = []
        self.depth = []
        self.url = url
        self.folder = folder
        print("init")
        if folder =="all" :
           self.fg_bg = get_images(url,'Fg-Bg') 
           self.fg_bg_mask = get_images(url,'Fg-Bg-Mask') 
           self.depth = get_images(url,'Fg-Bg') 
        elif (folder !="all" and folder=='Fg-Bg') :
           self.fg_bg = get_images(url,'Fg-Bg') 
        elif (folder !="all" and folder=='Fg-Bg-Mask') :
           self.fg_bg_mask = get_images(url,'Fg-Bg-Mask') 
        elif (folder !="all" and folder=='Depth') :
           self.depth = get_images(url,'Depth') 
        else :
           print("Path does not exisit",url+folder+'/')
    def __len__(self):
        if(self.folder=="all"):
          return len(self.fg_bg)
        elif (self.folder=='Fg-Bg') :
           return len(self.fg_bg)
        elif (self.folder=='Fg-Bg-Mask') :
           return len(self.fg_bg_mask) 
        elif (self.folder=='Depth') :
           return len(self.depth) 
        else :
           return -1
    def __getitem__(self, idx):
        if(self.folder=="all") :
          fg_bg = self.fg_bg[idx]
          fg_bg_mask = self.fg_bg_mask[idx]
          depth = self.depth[idx]
          return fg_bg,fg_bg_mask,depth
        elif (self.folder=='Fg-Bg') :
           fg_bg = self.fg_bg[idx]
           return fg_bg
        elif (self.folder=='Fg-Bg-Mask') :
           fg_bg_mask = self.fg_bg_mask[idx]
           return fg_bg_mask
        elif (self.folder=='Depth') :
           depth = self.depth[idx]
           return depth
        else :
           return None

In [0]:
from torch.utils.data import Dataset, random_split
from PIL import Image
import numpy as np
import torch
import os
import torchvision.transforms as transforms
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

In [10]:
from torch.utils.data import Dataset, random_split
import numpy as np
url = '/content/data_1/'
input_dataset = InputDataSet(url,'Fg-Bg')
train_split = 70
train_len = len(input_dataset)*train_split//100
test_len = len(input_dataset) - train_len 
train_set, val_set = random_split(input_dataset, [train_len, test_len])
print(len(train_set))
print(len(val_set))
train_dataset = DatasetFromSubset(train_set, transform=None)
test_dataset = DatasetFromSubset(val_set, transform=None)
print(len(train_dataset))
print(len(test_dataset))

init
Loading /content/data_1/Fg-Bg/
56000
24000
56000
24000


In [11]:
batch_size=256
num_workers=4
pin_memory=True
SEED = 1
cuda = torch.cuda.is_available()
print(cuda)
torch.manual_seed(SEED)
if cuda:
			torch.cuda.manual_seed(SEED)
dataloader_args = dict(shuffle=True, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory) if cuda else dict(shuffle=True, batch_size=64)

trainloader = torch.utils.data.DataLoader(train_dataset, **dataloader_args)
testloader = torch.utils.data.DataLoader(test_dataset, **dataloader_args)

False


In [14]:
print(trainloader.batch_size)
print(testloader.batch_size)

64
64


In [0]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1,dropout=0.0):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.dropout = dropout


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.bn2(self.conv2(out))
        out = F.dropout(out, p=self.dropout)
        out += self.shortcut(x)
        out = F.relu(out)
        out = F.dropout(out, p=self.dropout)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout=0.0):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.dropout = dropout

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout=self.dropout))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.dropout(out, p=self.dropout)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18(dropout=0, num_classes = 10):
    return ResNet(BasicBlock, [2,2,2,2],num_classes = num_classes , dropout=dropout)

In [18]:
!pip install torchsummary
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = ResNet18(num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
            Conv2d-3           [-1, 64, 64, 64]          36,864
       BatchNorm2d-4           [-1, 64, 64, 64]             128
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
        BasicBlock-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
           Conv2d-10           [-1, 64, 64, 64]          36,864
      BatchNorm2d-11           [-1, 64, 64, 64]             128
       BasicBlock-12           [-1, 64, 64, 64]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
      BatchNorm2d-14          [-1, 128,

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR,StepLR

optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay = 0.0001,nesterov = True ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, max_lr = 0.2, total_steps=None, epochs=30, steps_per_epoch=1, pct_start=1/3, anneal_strategy='linear', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=10.0,final_div_factor =10)